# 作業

1. 找一個資料集作訓練，方法不限
2. 表示你改進準確率的想法並實作
3. 9/2 開學前上傳

## Dataset

[Fruits Classification](https://www.kaggle.com/code/karimabdulnabi/fruits-classification-by-cnn)

10 Class:
- Apple
- Orange
- Avocado
- Kiwi
- Mango
- Pinenapple
- Strawberries
- Banana
- Cherry
-Watermelon

參考解答：這篇沒有說明，我對每個cell補上說明，再參數測試

[fruits classification by CNN 🥝🍇](https://www.kaggle.com/code/karimabdulnabi/fruits-classification-by-cnn)

# Code

## Import Library

### Environment

In [ ]:
# %pip install -r linux_requirements.txt
# %pip install -r mac_requirements.txt

In [ ]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import glob as gb
import os

## Data Preprocessing

### Load Data

In [ ]:
trainpath = "./MY_data/train/"
testpath = "./MY_data/test/"
predpath = "./MY_data/predict/"

In [ ]:
for folder in os.listdir(trainpath):
    files = gb.glob(pathname=str(trainpath+folder + "/*.jpeg"))
    print(f"for training data , found {len(files)} in folder {folder}")

In [ ]:
for folder in os.listdir(testpath):
    files = gb.glob(pathname=str(testpath+folder + "/*.jpeg"))
    print(f"for testing data , found {len(files)} in folder {folder}")

### Encode Class

In [ ]:
code_train = {"Apple" : 0, "avocado" :1 , "Banana" :2 , "cherry" :3 , "kiwi" :4 , "mango" :5 , "orange" :6 , "pinenapple" :7 ,
        "strawberries" :8 , "watermelon" :9 }

def getcode_train(n):
    for x , y in code_train.items():
        if n==y:
            return x

# check ording of the classes
print(f'Apple code is : {code_train["Apple"]}')

In [ ]:
code_test = {"apple" : 0, "avocado" :1 , "banana" :2 , "cherry" :3 , "kiwi" :4 , "mango" :5 , "orange" :6 , "pinenapple" :7 ,
        "stawberries" :8 , "watermelon" :9 }

def getcode_test(n):
    for x , y in code_test.items():
        if n==y:
            return x
        
# check ording of the classes
print(f"manago code is : {code_test['mango']}")

### Image Size

In [ ]:
size = []
for folder in os.listdir(trainpath):
    files = gb.glob(pathname=str(trainpath+folder + "/*.jpeg"))
    for file in files:
        image = plt.imread(file)
        size.append(image.shape)
        
df = pd.Series(size)
pd.DataFrame(df.value_counts())#[:60]

In [ ]:
size = []
for folder in os.listdir(testpath):
    files = gb.glob(pathname=str(testpath+folder + "/*.jpeg"))
    for file in files:
        image = plt.imread(file)
        size.append(image.shape)
        
df = pd.Series(size)
pd.DataFrame(df.value_counts())

### Resize

In [ ]:
s = 150

#### Testing data Resize

In [ ]:
x_test = []
y_test = []
for folder in os.listdir(testpath):
    files = gb.glob(pathname=str(testpath+folder + "/*.jpeg"))
    for file in files:
        image = cv.imread(file)
        image_array = cv.resize(image , (s,s))
        x_test.append(list(image_array))
        y_test.append(code_test[folder])

In [ ]:
l = len(x_test)

In [ ]:
plt.figure(figsize=(20 , 20))
for n , i in enumerate(list(np.random.randint(0 , l ,36 ))) :
    plt.subplot(6 , 6 , n+1)
    plt.imshow(x_test[i])
    plt.axis("off")
    plt.title(getcode_test(y_test[i]))

#### Training data Resize

In [ ]:
x_train = []
y_train = []
for folder in os.listdir(trainpath):
    files = gb.glob(pathname=str(trainpath+folder + "/*.jpeg"))
    for file in files:
        image = cv.imread(file)
        if image is None:
            continue
        image_array = cv.resize(image , (s,s))
        x_train.append(list(image_array))
        y_train.append(code_train[folder])

In [ ]:
plt.figure(figsize=(20 , 20))
for n , i in enumerate(list(np.random.randint(0 , l ,36 ))) :
    plt.subplot(6 , 6 , n+1)
    plt.imshow(x_train[i])
    plt.axis("off")
    plt.title(getcode_train(y_train[i]))

In [ ]:
x_pred = []
files = gb.glob(pathname=str(predpath +"/*.jpeg"))
for file in files:
    image = cv.imread(file)
    image_array = cv.resize(image , (s , s))
    x_pred.append(list(image_array))

In [ ]:
len(x_pred)

In [ ]:
plt.figure(figsize=(20 , 20))
for o , p in enumerate(list(np.random.randint(0 , 48 ,36))):
    plt.subplot(6 , 6 , o+1)
    plt.imshow(x_pred[p])
    plt.axis("off")

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_pred_array = np.array(x_pred)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
print(x_test.shape)

## Model

### CNN Model Architecture Setup

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(200 , kernel_size = (5 , 5) , activation="relu" , input_shape=(s, s , 3)) , 
    keras.layers.MaxPool2D(5 , 5) ,
    keras.layers.Conv2D(100 ,kernel_size = (5 , 5), activation="relu") , 
    keras.layers.MaxPool2D(5 , 5) ,
    keras.layers.Flatten() , 
    keras.layers.Dense(50 , activation="relu") ,
    keras.layers.Dropout(rate=0.3) , 
    keras.layers.Dense(10 , activation="softmax") ,
])

In [ ]:
model.compile(optimizer="adam" , loss="sparse_categorical_crossentropy" , metrics=["accuracy"])

In [ ]:
model.summary()

### Train Model

In [ ]:
model.fit(x_train , y_train , epochs=10 , batch_size = 64)

### Evaluation

In [ ]:
modelloss , modelaccuracy = model.evaluate(x_test , y_test)

### Prediction

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_result = model.predict(x_pred_array)
y_result.shape

In [ ]:
plt.figure(figsize=(20 , 20))
for n , i in enumerate(list(np.random.randint(0 , 48 , 36))):
    plt.subplot(6 , 6 , n+1)
    plt.imshow(x_pred[i])
    plt.axis("off")
    plt.title(getcode_train(np.argmax(y_result[i])))

# 嘗試優化

## Model

### CNN Model Architecture Setup

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(250 , kernel_size = (3 , 3) , activation="relu" , input_shape=(s, s , 3)) , 
    keras.layers.Conv2D(230 , kernel_size = (3 , 3) , activation="relu") ,
    keras.layers.Conv2D(200 , kernel_size = (3 , 3) , activation="relu") , 
    keras.layers.MaxPool2D(6 , 6) ,
    keras.layers.Conv2D(180 ,kernel_size = (3 , 3), activation="relu") , 
    keras.layers.Conv2D(100 ,kernel_size = (3 , 3), activation="relu") , 
    keras.layers.Conv2D(80 , kernel_size = (3 , 3) , activation="relu") ,
    keras.layers.MaxPool2D(6 , 6) ,
    keras.layers.Flatten() , 
    keras.layers.Dense(120 , activation="relu") ,
    keras.layers.Dense(100 , activation="relu") ,
    keras.layers.Dense(50 , activation="relu") ,
    keras.layers.Dropout(rate=0.3) , 
    keras.layers.Dense(10 , activation="softmax") ,
])

In [ ]:
model.compile(optimizer="adam" , loss="sparse_categorical_crossentropy" , metrics=["accuracy"])

In [ ]:
model.summary()

### Train Model

In [ ]:
model.fit(x_train , y_train , epochs=30 , batch_size = 64)

In [ ]:
modelloss , modelaccuracy = model.evaluate(x_test , y_test)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_result = model.predict(x_pred_array)
y_result.shape

In [ ]:
plt.figure(figsize=(20 , 20))
for n , i in enumerate(list(np.random.randint(0 , 48 , 36))):
    plt.subplot(6 , 6 , n+1)
    plt.imshow(x_pred[i])
    plt.axis("off")
    plt.title(getcode_train(np.argmax(y_result[i])))